<h2>Capstone Week 4 Project - Data</h2>

#### Import libraries required to complete task

In [1]:
import csv
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

In [2]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         235 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2021.1.19-~ --> conda-forge::ca-certificates-

### Download and Explore Dataset

read csv file of Houston Wards and Houston Heights Lat and Lon figures

In [10]:
#read csv file
Hou_wards_df = pd.read_csv("Houston_Wards.csv")
Hou_wards_df

,Neighborhood,Latitude,Longitude
0,First Ward,29.763800,-95.359500
1,Second Ward,29.751100,-95.324000
2,Greater Third Ward,29.727800,-95.349900
3,Fourth Ward,29.757900,-95.380900
4,Greater Fifth Ward,29.776100,-95.326500
5,Old Sixth Ward,29.766339,-95.378272
6,Houston Heights,29.798100,-95.397900


In [12]:
lat = []
lng = []

def getLatLng(row):
    geolocator = Nominatim(user_agent='foursquare')
    print(row[0]+', '+row[1])
    location = geolocator.geocode(row[0]+', '+row[1])
    if location != None:
        lat.append(location.latitude)
        lng.append(location.longitude)
    else:
        lat.append(None)
        lng.append(None)

In [13]:
#import geocoder
from geopy.geocoders import Nominatim

address = 'Houston, Tx'
geolocator = Nominatim(user_agent = "hou_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coords of Houston are {}, {}.'.format(latitude, longitude))

The geographical coords of Houston are 29.7589382, -95.3676974.


Map of Houston with the six Wards and Houston Heights

In [255]:
# create map of New York using latitude and longitude values
map_Hou = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(Hou_wards_df['Latitude'], Hou_wards_df['Longitude'], Hou_wards_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Hou)  

map_Hou

Intialize FourSquare credentials

In [16]:
CLIENT_ID = 'GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ' # your Foursquare ID
CLIENT_SECRET = 'GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ
CLIENT_SECRET:GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH


First Ward

In [17]:
neighborhood_lat_1W = Hou_wards_df.loc[0, 'Latitude']
neighborhood_lon_1W = Hou_wards_df.loc[0, 'Longitude']
neighborhood_nei_1W = Hou_wards_df.loc[0, 'Neighborhood']

print('Lat and Lon values of {} are {}, {}.'.format(neighborhood_nei_1W, neighborhood_lat_1W, neighborhood_lon_1W))

Lat and Lon values of First Ward are 29.7638, -95.3595.


In [18]:
LIMIT = 100 #radius
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat_1W, 
    neighborhood_lon_1W, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ&client_secret=GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH&v=20180605&ll=29.7638,-95.3595&radius=500&limit=100'

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60760bd5ecec8820aea47c9c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Houston',
  'headerFullLocation': 'Downtown Houston, Houston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 70,
  'suggestedBounds': {'ne': {'lat': 29.768300004500006,
    'lng': -95.3543258153177},
   'sw': {'lat': 29.759299995499994, 'lng': -95.3646741846823}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '541c2faf498e5208ed83297e',
       'name': 'Houston Watch Co.',
       'location': {'address': '913 Franklin St',
        'crossStreet': 'Main St.',
        'lat': 29.763274017872376,
        'lng'

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues['Ward'] = "First Ward"
nearby_venues_1W = nearby_venues
Hou_1W_csv = nearby_venues_1W.to_csv('Hou_1W.csv', index=False)
nearby_venues_1W

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,Ward
0,Houston Watch Co.,Bar,29.763274,-95.360661,First Ward
1,Craft Beer Cellar Houston,Beer Store,29.763562,-95.360812,First Ward
2,La Carafe,Wine Bar,29.763041,-95.361904,First Ward
3,Hearsay Gastro Lounge,New American Restaurant,29.762867,-95.361763,First Ward
4,Public Services Wine & Whiskey,Wine Bar,29.763509,-95.361135,First Ward
5,La Fisheria,Mexican Restaurant,29.763684,-95.362186,First Ward
6,Captain Foxheart's Bad News Bar & Spirits Lodge,Cocktail Bar,29.761987,-95.361030,First Ward
7,Boomtown Coffee,Coffee Shop,29.762283,-95.360976,First Ward
8,Club Proxima (NEXT),Nightclub,29.763413,-95.359820,First Ward
9,Market Square Park,Park,29.762556,-95.362137,First Ward


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

70 venues were returned by Foursquare.


Second Ward Data

In [23]:
neighborhood_lat_2W = Hou_wards_df.loc[1, 'Latitude']
neighborhood_lon_2W = Hou_wards_df.loc[1, 'Longitude']
neighborhood_nei_2W = Hou_wards_df.loc[1, 'Neighborhood']

print('Lat and Lon values of {} are {}, {}.'.format(neighborhood_nei_2W, neighborhood_lat_2W, neighborhood_lon_2W))

Lat and Lon values of Second Ward are 29.7511, -95.324.


In [24]:
LIMIT = 100 #radius
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat_2W, 
    neighborhood_lon_2W, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ&client_secret=GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH&v=20180605&ll=29.7511,-95.324&radius=500&limit=100'

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60760bd60a581c7f303b1121'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Second Ward',
  'headerFullLocation': 'Second Ward, Houston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 11,
  'suggestedBounds': {'ne': {'lat': 29.755600004500007,
    'lng': -95.31882647097753},
   'sw': {'lat': 29.746599995499995, 'lng': -95.32917352902247}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50afbb77e4b0211a31a22d82',
       'name': 'Texas Mattress Makers',
       'location': {'address': '4619 Navigation Blvd',
        'crossStreet': 'Lockwood Dr',
        'lat': 29.752096010409602,
        'lng': -95.3237248121643,
        'labele

In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues['Ward'] = "Second Ward"
nearby_venues_2W = nearby_venues
Hou_2W_csv = nearby_venues_2W.to_csv('Hou_2W.csv', index=False)
nearby_venues_2W

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,Ward
0,Texas Mattress Makers,Mattress Store,29.752096,-95.323725,Second Ward
1,The White Swan,Bar,29.752043,-95.326671,Second Ward
2,"""La Silla"" Pollos asados al carbon",Mexican Restaurant,29.748268,-95.327063,Second Ward
3,Papa John's Pizza,Pizza Place,29.747859,-95.327196,Second Ward
4,McDonald's,Fast Food Restaurant,29.751424,-95.322771,Second Ward
5,Shell,Gas Station,29.751420,-95.323254,Second Ward
6,Living Designs Furniture,Furniture / Home Store,29.752608,-95.324769,Second Ward
7,Taqueria Chabelita,Taco Place,29.750229,-95.321641,Second Ward
8,Southern Crushed Concrete / Asphalt,Construction & Landscaping,29.753227,-95.320544,Second Ward
9,Garcias Party Store,Event Service,29.747856,-95.327080,Second Ward


Third Ward Data

In [28]:
neighborhood_lat_3W = Hou_wards_df.loc[2, 'Latitude']
neighborhood_lon_3W = Hou_wards_df.loc[2, 'Longitude']
neighborhood_nei_3W = Hou_wards_df.loc[2, 'Neighborhood']

print('Lat and Lon values of {} are {}, {}.'.format(neighborhood_nei_3W, neighborhood_lat_3W, neighborhood_lon_3W))

Lat and Lon values of Greater Third Ward are 29.7278, -95.3499.


In [29]:
# type your answer here
#manhattan_data.loc[0:100, 'Neighborhood']

LIMIT = 100 #radius
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat_3W, 
    neighborhood_lon_3W, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ&client_secret=GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH&v=20180605&ll=29.7278,-95.3499&radius=500&limit=100'

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60760bd61149e00d940203bd'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Greater Third Ward',
  'headerFullLocation': 'Greater Third Ward, Houston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 14,
  'suggestedBounds': {'ne': {'lat': 29.732300004500004,
    'lng': -95.34472767278837},
   'sw': {'lat': 29.723299995499993, 'lng': -95.35507232721164}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5aa57b32018cbb71ad85cd74',
       'name': 'Fertitta Center',
       'location': {'lat': 29.724703281852417,
        'lng': -95.34715537168006,
        'labeledLatLngs': [{'label': 'display',
          'lat': 29.724703281852417,
  

In [31]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues['Ward'] = "Third Ward"
nearby_venues_3W = nearby_venues
Hou_3W_csv = nearby_venues_3W.to_csv('Hou_3W.csv', index=False)
nearby_venues_3W

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,Ward
0,Fertitta Center,Basketball Stadium,29.724703,-95.347155,Third Ward
1,Hoàng Sandwich,Vietnamese Restaurant,29.728033,-95.349119,Third Ward
2,The Library Coffee & Wine House,Coffee Shop,29.732206,-95.348890,Third Ward
3,Cream Burger,Burger Joint,29.728179,-95.350544,Third Ward
4,Taco Bell,Fast Food Restaurant,29.726042,-95.350469,Third Ward
5,Domino's Pizza,Pizza Place,29.725710,-95.350465,Third Ward
6,METRORail Elgin / Third Ward Station,Light Rail Station,29.728384,-95.349883,Third Ward
7,The Fish Place,Seafood Restaurant,29.728131,-95.348897,Third Ward
8,Cougar Softball Stadium,Baseball Field,29.726923,-95.348848,Third Ward
9,The Breakfast Klub,Breakfast Spot,29.728001,-95.349030,Third Ward


Fourth Ward Data

In [33]:
neighborhood_lat_4W = Hou_wards_df.loc[3, 'Latitude']
neighborhood_lon_4W = Hou_wards_df.loc[3, 'Longitude']
neighborhood_nei_4W = Hou_wards_df.loc[3, 'Neighborhood']

print('Lat and Lon values of {} are {}, {}.'.format(neighborhood_nei_4W, neighborhood_lat_4W, neighborhood_lon_4W))

Lat and Lon values of Fourth Ward are 29.7579, -95.3809.


In [34]:
LIMIT = 100 #radius
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat_4W, 
    neighborhood_lon_4W, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ&client_secret=GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH&v=20180605&ll=29.7579,-95.3809&radius=500&limit=100'

In [35]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60760bd70a581c7f303b1398'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Fourth Ward',
  'headerFullLocation': 'Fourth Ward, Houston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 29.762400004500005,
    'lng': -95.37572611996785},
   'sw': {'lat': 29.753399995499993, 'lng': -95.38607388003214}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bff00bcca1920a16f32ec81',
       'name': 'Eleanor Tinsley Park',
       'location': {'address': '500 Allen Parkway',
        'lat': 29.761439843452138,
        'lng': -95.37927095656195,
        'labeledLatLngs': [{'label': 'display',
         

In [36]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues['Ward'] = "Fourth Ward"
nearby_venues_4W = nearby_venues
Hou_4W_csv = nearby_venues_4W.to_csv('Hou_4W.csv', index=False)
nearby_venues_4W

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,Ward
0,Eleanor Tinsley Park,Park,29.761440,-95.379271,Fourth Ward
1,Paper Street Crossfit,Gym,29.757435,-95.385846,Fourth Ward
2,Lucio's BYOB,Bar,29.758326,-95.385591,Fourth Ward
3,Extreme Custom & Classic Cars,Auto Garage,29.757708,-95.385589,Fourth Ward
4,Best Western Plus Downtown Inn & Suites,Hotel,29.757376,-95.377440,Fourth Ward
5,Tony's Corner Pocket,Gay Bar,29.757605,-95.376829,Fourth Ward
6,Reflection Pool,Park,29.759820,-95.383963,Fourth Ward
7,Bludorn,Restaurant,29.758396,-95.385637,Fourth Ward
8,Buffalo Bayou,River,29.762049,-95.379603,Fourth Ward


Fifth Ward Data

In [38]:
neighborhood_lat_5W = Hou_wards_df.loc[4, 'Latitude']
neighborhood_lon_5W = Hou_wards_df.loc[4, 'Longitude']
neighborhood_nei_5W = Hou_wards_df.loc[4, 'Neighborhood']

print('Lat and Lon values of {} are {}, {}.'.format(neighborhood_nei_5W, neighborhood_lat_5W, neighborhood_lon_5W))

Lat and Lon values of Greater Fifth Ward are 29.7761, -95.3265.


In [39]:
LIMIT = 100 #radius
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat_5W, 
    neighborhood_lon_5W, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ&client_secret=GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH&v=20180605&ll=29.7761,-95.3265&radius=500&limit=100'

In [40]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60760bd7e67e012e704e9475'},
 'response': {'headerLocation': 'Greater Fifth Ward',
  'headerFullLocation': 'Greater Fifth Ward, Houston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 29.780600004500005,
    'lng': -95.32132517990773},
   'sw': {'lat': 29.771599995499994, 'lng': -95.33167482009226}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c3e57706faac9b6a7230e76',
       'name': 'Frenchys Chicken',
       'location': {'address': '4403 East Fwy',
        'lat': 29.773792967574607,
        'lng': -95.32607797848243,
        'labeledLatLngs': [{'label': 'display',
          'lat': 29.773792967574607,
          'lng': -95.32607797848243},
         {'label': 'entrance', 'la

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [42]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues['Ward'] = "Fifth Ward"
nearby_venues_5W = nearby_venues
Hou_5W_csv = nearby_venues_5W.to_csv('Hou_5W.csv', index=False)
nearby_venues_5W

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,Ward
0,Frenchys Chicken,Fried Chicken Joint,29.773793,-95.326078,Fifth Ward
1,Reagan Shoe Repair,Other Repair Shop,29.776241,-95.327690,Fifth Ward
2,FIFTH WARD FINEST,Park,29.777791,-95.325061,Fifth Ward
3,I10 and Waco,Intersection,29.773314,-95.326310,Fifth Ward


Sixth Ward

In [43]:
neighborhood_lat_6W = Hou_wards_df.loc[5, 'Latitude']
neighborhood_lon_6W = Hou_wards_df.loc[5, 'Longitude']
neighborhood_nei_6W = Hou_wards_df.loc[5, 'Neighborhood']

print('Lat and Lon values of {} are {}, {}.'.format(neighborhood_nei_6W, neighborhood_lat_6W, neighborhood_lon_6W))

Lat and Lon values of Old Sixth Ward are 29.7663391, -95.3782719.


In [44]:
LIMIT = 100 #radius
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat_6W, 
    neighborhood_lon_6W, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ&client_secret=GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH&v=20180605&ll=29.7663391,-95.3782719&radius=500&limit=100'

In [45]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60760bd8727d6301d8009b53'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Sixth Ward',
  'headerFullLocation': 'Sixth Ward, Houston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 19,
  'suggestedBounds': {'ne': {'lat': 29.770839104500006,
    'lng': -95.37309758418176},
   'sw': {'lat': 29.761839095499994, 'lng': -95.38344621581824}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5033e2e3e4b0ee26f2d6aa57',
       'name': 'Julep',
       'location': {'address': '1919 Washington Ave.',
        'crossStreet': 'White St.',
        'lat': 29.767723245110254,
        'lng': -95.3780814962

In [46]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [47]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues['Ward'] = "Sixth Ward"
nearby_venues_6W = nearby_venues
Hou_6W_csv = nearby_venues_6W.to_csv('Hou_6W.csv', index=False)
nearby_venues_6W

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,Ward
0,Julep,Cocktail Bar,29.767723,-95.378081,Sixth Ward
1,Tacodeli,Taco Place,29.767928,-95.377333,Sixth Ward
2,Gus's World Famous Fried Chicken,Fried Chicken Joint,29.767542,-95.376473,Sixth Ward
3,Platypus Brewing,Brewery,29.768071,-95.377502,Sixth Ward
4,B & B Butchers,Steakhouse,29.767933,-95.376494,Sixth Ward
5,Catalina Coffee,Coffee Shop,29.767968,-95.380662,Sixth Ward
6,Washington Gym,Gym,29.767661,-95.378145,Sixth Ward
7,Henderson Heights,Gastropub,29.767525,-95.379302,Sixth Ward
8,Taqueria Barba,Food Truck,29.767813,-95.380799,Sixth Ward
9,Old Sixth Ward Historical District,Historic Site,29.764275,-95.378041,Sixth Ward


Houston Heights

In [48]:
neighborhood_lat_HH = Hou_wards_df.loc[6, 'Latitude']
neighborhood_lon_HH = Hou_wards_df.loc[6, 'Longitude']
neighborhood_nei_HH = Hou_wards_df.loc[6, 'Neighborhood']

print('Lat and Lon values of {} are {}, {}.'.format(neighborhood_nei_HH, neighborhood_lat_HH, neighborhood_lon_HH))

Lat and Lon values of Houston Heights are 29.7981, -95.3979.


In [49]:
LIMIT = 100 #radius
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat_HH, 
    neighborhood_lon_HH, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GZXKFFU23VOF1LUQVOSLG0KJC3FME0RXXQEG12G5D4MHR4EQ&client_secret=GAUOXYSRR30P1DFRNCE2Y3GOHH5CHCLRNKIWPTDJMUYMUUIH&v=20180605&ll=29.7981,-95.3979&radius=500&limit=100'

In [50]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60760bd8391d4c70417bb15a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Greater Heights',
  'headerFullLocation': 'Greater Heights, Houston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': 29.802600004500007,
    'lng': -95.39272404241783},
   'sw': {'lat': 29.793599995499996, 'lng': -95.40307595758219}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '55f4747d498e264103800fc3',
       'name': 'The Wilde Collection',
       'location': {'address': '1446 Yale St',
        'crossStreet': '15th',
        'lat': 29.797919585722735,
        'lng': -95.39888963052749,
        'labeledLatLng

In [51]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [52]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues['Ward'] = "Hou Heights"
nearby_venues_HH = nearby_venues
Hou_HH_csv = nearby_venues_HH.to_csv('Hou_HH.csv', index=False)
nearby_venues_HH

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,Ward
0,The Wilde Collection,Gift Shop,29.797920,-95.398890,Hou Heights
1,Cosa Bella Salon & Day Spa,Salon / Barbershop,29.799587,-95.399416,Hou Heights
2,La Paz Day Spa,Spa,29.796223,-95.398476,Hou Heights
3,Pink's Pizza,Pizza Place,29.796407,-95.398242,Hou Heights
4,Heights Running Trail,Trail,29.802266,-95.398002,Hou Heights
5,Opera In The Heights,Opera House,29.801161,-95.398474,Hou Heights
6,Down House,New American Restaurant,29.802214,-95.399355,Hou Heights
7,Marmion Park,Park,29.802301,-95.397266,Hou Heights
8,Lambert Hall,Performing Arts Venue,29.801292,-95.398201,Hou Heights
9,Claire's School Of Dance,Dance Studio,29.793875,-95.398544,Hou Heights


Combined Wards and Hou Heights DF

In [59]:
csv_file_list = ["Hou_1W.csv","Hou_2W.csv","Hou_3W.csv","Hou_4W.csv","Hou_5W.csv","Hou_6W.csv","Hou_HH.csv"]
list_of_dataframes = []
for filename in csv_file_list:
    list_of_dataframes.append(pd.read_csv(filename))
merged_df = pd.concat(list_of_dataframes)

In [60]:
print(merged_df.shape[0],"venues within the Houston Wards and Hou Heights")
merged_df.reset_index(inplace=True)

137 venues within the Houston Wards and Hou Heights


In [102]:
#merged_df.groupby('Ward').count()
Hou_merged = merged_df[merged_df["categories"] != "Intersection"].reset_index(drop=True)
Hou_merged_1 = Hou_merged[Hou_merged["categories"] != "Park"].reset_index(drop=True)
Hou_merged_2 = Hou_merged_1[Hou_merged_1["categories"] != "Trail"].reset_index(drop=True)
#df_new = df.drop(df[(df['col_1'] == 1.0) & (df['col_2'] == 0.0)].index) #clearner method to consider

#Hou_merged_2.reset_index(drop=True) #display full table
Hou_WardHH_DF = Hou_merged_2.drop(['index'], axis=1)
Hou_WardHH_DF[:5]

,name,categories,lat,lng,Ward
0,Houston Watch Co.,Bar,29.763274,-95.360661,First Ward
1,Craft Beer Cellar Houston,Beer Store,29.763562,-95.360812,First Ward
2,La Carafe,Wine Bar,29.763041,-95.361904,First Ward
3,Hearsay Gastro Lounge,New American Restaurant,29.762867,-95.361763,First Ward
4,Public Services Wine & Whiskey,Wine Bar,29.763509,-95.361135,First Ward


Analyze each Ward

In [103]:
# move neighborhood column to the first column
Hou_WardHH_DF_onehot = pd.get_dummies(Hou_WardHH_DF[['categories']], prefix="", prefix_sep="")

Hou_WardHH_DF_onehot['Ward'] = Hou_WardHH_DF['Ward']

fixed_columns = [Hou_WardHH_DF_onehot.columns[-1]] + list(Hou_WardHH_DF_onehot.columns[:-1])
Hou_WardHH_DF_onehot = Hou_WardHH_DF_onehot[fixed_columns]

Hou_WardHH_DF_onehot[:5]

,Ward,American Restaurant,Asian Restaurant,Auto Garage,BBQ Joint,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Garden,Beer Store,Bistro,Breakfast Spot,Brewery,Burger Joint,Café,Clothing Store,Cocktail Bar,Coffee Shop,Construction & Landscaping,Dance Studio,Dive Bar,Event Service,Fast Food Restaurant,Food Court,Food Service,Food Truck,Frame Store,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Gym,Historic Site,Hotel,Japanese Restaurant,Jazz Club,Latin American Restaurant,Light Rail Station,Lounge,Mattress Store,Mexican Restaurant,Mosque,New American Restaurant,Nightclub,Opera House,Other Repair Shop,Performing Arts Venue,Pharmacy,Pizza Place,Pub,Restaurant,River,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Sports Bar,Steakhouse,Taco Place,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant,Wine Bar
0,First Ward,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,First Ward,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,First Ward,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,First Ward,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,First Ward,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [104]:
Hou_WardHH_DF_onehot.shape[0]

130

In [105]:
Hou_WardHH_DF_grouped = Hou_WardHH_DF_onehot.groupby('Ward').mean().reset_index()
Hou_WardHH_DF_grouped

,Ward,American Restaurant,Asian Restaurant,Auto Garage,BBQ Joint,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Garden,Beer Store,Bistro,Breakfast Spot,Brewery,Burger Joint,Café,Clothing Store,Cocktail Bar,Coffee Shop,Construction & Landscaping,Dance Studio,Dive Bar,Event Service,Fast Food Restaurant,Food Court,Food Service,Food Truck,Frame Store,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Gym,Historic Site,Hotel,Japanese Restaurant,Jazz Club,Latin American Restaurant,Light Rail Station,Lounge,Mattress Store,Mexican Restaurant,Mosque,New American Restaurant,Nightclub,Opera House,Other Repair Shop,Performing Arts Venue,Pharmacy,Pizza Place,Pub,Restaurant,River,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Sports Bar,Steakhouse,Taco Place,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant,Wine Bar
0,Fifth Ward,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.5,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,First Ward,0.014493,0.000000,0.000000,0.014493,0.115942,0.000000,0.000000,0.000000,0.014493,0.014493,0.014493,0.000000,0.000000,0.057971,0.028986,0.000000,0.115942,0.057971,0.000000,0.014493,0.000000,0.000000,0.000000,0.014493,0.014493,0.014493,0.000000,0.000000,0.000000,0.000000,0.028986,0.000000,0.000,0.014493,0.000000,0.000000,0.028986,0.000000,0.014493,0.028986,0.000000,0.043478,0.000000,0.043478,0.014493,0.028986,0.028986,0.000,0.0,0.000,0.014493,0.028986,0.014493,0.000000,0.000000,0.000,0.028986,0.000000,0.014493,0.043478,0.000,0.014493,0.000000,0.014493,0.014493,0.000000,0.000000,0.028986
2,Fourth Ward,0.000000,0.000000,0.142857,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000,0.000000,0.142857,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000,0.000000,0.000000,0.000000,0.142857,0.142857,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Hou Heights,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.125,0.0,0.125,0.000000,0.125000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Second Ward,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.090909,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.090909,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.090909,0.000000,0.000000,0.000000,0.000,0.0,0.000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000
5,Sixth Ward,0.000000,0.00000

In [137]:
num_top_venues = 5

for hood in Hou_WardHH_DF_grouped['Ward']:
    print("----"+hood+"----")
    temp = Hou_WardHH_DF_grouped[Hou_WardHH_DF_grouped['Ward'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Fifth Ward----
                   venue  freq
0      Other Repair Shop   0.5
1    Fried Chicken Joint   0.5
2    American Restaurant   0.0
3     Mexican Restaurant   0.0
4  Performing Arts Venue   0.0


----First Ward----
          venue  freq
0           Bar  0.12
1  Cocktail Bar  0.12
2  Burger Joint  0.06
3   Coffee Shop  0.06
4        Lounge  0.04


----Fourth Ward----
        venue  freq
0         Gym  0.14
1       River  0.14
2     Gay Bar  0.14
3       Hotel  0.14
4  Restaurant  0.14


----Hou Heights----
                     venue  freq
0              Pizza Place  0.12
1                      Spa  0.12
2              Opera House  0.12
3       Salon / Barbershop  0.12
4  New American Restaurant  0.12


----Second Ward----
                        venue  freq
0  Construction & Landscaping  0.09
1          Mexican Restaurant  0.09
2        Fast Food Restaurant  0.09
3                Burger Joint  0.09
4      Furniture / Home Store  0.09


----Sixth Ward----
                     

In [202]:
Hou_WardHH_DF = Hou_WardHH_DF[['Ward','name', 'categories', 'lat', 'lng']]
Hou_WardHH_DF[:5]

,Ward,name,categories,lat,lng
0,First Ward,Houston Watch Co.,Bar,29.763274,-95.360661
1,First Ward,Craft Beer Cellar Houston,Beer Store,29.763562,-95.360812
2,First Ward,La Carafe,Wine Bar,29.763041,-95.361904
3,First Ward,Hearsay Gastro Lounge,New American Restaurant,29.762867,-95.361763
4,First Ward,Public Services Wine & Whiskey,Wine Bar,29.763509,-95.361135


In [208]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

In [213]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Ward']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Ward'] = Hou_WardHH_DF_grouped['Ward']

for ind in np.arange(Hou_WardHH_DF_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Hou_WardHH_DF_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Ward,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Fifth Ward,Other Repair Shop,Fried Chicken Joint,Wine Bar,Food Court,Construction & Landscaping
1,First Ward,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
2,Fourth Ward,Gay Bar,Auto Garage,Gym,Bar,Hotel
3,Hou Heights,Performing Arts Venue,Salon / Barbershop,Gift Shop,New American Restaurant,Pizza Place
4,Second Ward,Construction & Landscaping,Mattress Store,Gas Station,Furniture / Home Store,Taco Place
5,Sixth Ward,Clothing Store,Brewery,Gastropub,Fried Chicken Joint,Food Truck
6,Third Ward,Fast Food Restaurant,Frame Store,Burger Joint,Asian Restaurant,Vietnamese Restaurant


In [214]:
# set number of clusters
kclusters = 5

Hou_WardHH_DF_grouped_clustering = Hou_WardHH_DF_grouped.drop('Ward', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Hou_WardHH_DF_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 2, 4, 0, 0, 3], dtype=int32)

In [215]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [216]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Hou_wards_df_merged = Hou_WardHH_DF

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Hou_wards_df_merged = Hou_wards_df_merged.join(neighborhoods_venues_sorted.set_index('Ward'), on='Ward')

Hou_wards_df_merged # check the last columns!

,Ward,name,categories,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,First Ward,Houston Watch Co.,Bar,29.763274,-95.360661,0,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
1,First Ward,Craft Beer Cellar Houston,Beer Store,29.763562,-95.360812,0,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
2,First Ward,La Carafe,Wine Bar,29.763041,-95.361904,0,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
3,First Ward,Hearsay Gastro Lounge,New American Restaurant,29.762867,-95.361763,0,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
4,First Ward,Public Services Wine & Whiskey,Wine Bar,29.763509,-95.361135,0,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
5,First Ward,La Fisheria,Mexican Restaurant,29.763684,-95.362186,0,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
6,First Ward,Captain Foxheart's Bad News Bar & Spirits Lodge,Cocktail Bar,29.761987,-95.361030,0,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
7,First Ward,Boomtown Coffee,Coffee Shop,29.762283,-95.360976,0,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
8,First Ward,Club Proxima (NEXT),Nightclub,29.763413,-95.359820,0,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
9,First Ward,The Original OKRA Charity Saloon,Cocktail Bar,29.762345,-95.361249,0,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge


In [218]:
Hou_wards_df_merged[:5]

,Ward,name,categories,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,First Ward,Houston Watch Co.,Bar,29.763274,-95.360661,0,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
1,First Ward,Craft Beer Cellar Houston,Beer Store,29.763562,-95.360812,0,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
2,First Ward,La Carafe,Wine Bar,29.763041,-95.361904,0,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
3,First Ward,Hearsay Gastro Lounge,New American Restaurant,29.762867,-95.361763,0,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
4,First Ward,Public Services Wine & Whiskey,Wine Bar,29.763509,-95.361135,0,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge


In [256]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Hou_wards_df_merged['lat'], Hou_wards_df_merged['lng'], Hou_wards_df_merged['Ward'], Hou_wards_df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1

In [251]:
Hou_wards_df_merged.loc[Hou_wards_df_merged['Cluster Labels'] == 0, Hou_wards_df_merged.columns[[0] + list(range(6, Hou_wards_df_merged.shape[1]))]]

,Ward,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,First Ward,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
1,First Ward,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
2,First Ward,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
3,First Ward,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
4,First Ward,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
5,First Ward,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
6,First Ward,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
7,First Ward,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
8,First Ward,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge
9,First Ward,Bar,Cocktail Bar,Coffee Shop,Burger Joint,Lounge


Cluster 2

In [250]:
Hou_wards_df_merged.loc[Hou_wards_df_merged['Cluster Labels'] == 1, Hou_wards_df_merged.columns[[0] + list(range(6, Hou_wards_df_merged.shape[1]))]]

,Ward,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
101,Fifth Ward,Other Repair Shop,Fried Chicken Joint,Wine Bar,Food Court,Construction & Landscaping
102,Fifth Ward,Other Repair Shop,Fried Chicken Joint,Wine Bar,Food Court,Construction & Landscaping


Cluster 3

In [249]:
Hou_wards_df_merged.loc[Hou_wards_df_merged['Cluster Labels'] == 2, Hou_wards_df_merged.columns[[0] + list(range(6, Hou_wards_df_merged.shape[1]))]]

,Ward,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
94,Fourth Ward,Gay Bar,Auto Garage,Gym,Bar,Hotel
95,Fourth Ward,Gay Bar,Auto Garage,Gym,Bar,Hotel
96,Fourth Ward,Gay Bar,Auto Garage,Gym,Bar,Hotel
97,Fourth Ward,Gay Bar,Auto Garage,Gym,Bar,Hotel
98,Fourth Ward,Gay Bar,Auto Garage,Gym,Bar,Hotel
99,Fourth Ward,Gay Bar,Auto Garage,Gym,Bar,Hotel
100,Fourth Ward,Gay Bar,Auto Garage,Gym,Bar,Hotel


Cluster 4

In [248]:
Hou_wards_df_merged.loc[Hou_wards_df_merged['Cluster Labels'] == 3, Hou_wards_df_merged.columns[[0] + list(range(6, Hou_wards_df_merged.shape[1]))]]

,Ward,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
80,Third Ward,Fast Food Restaurant,Frame Store,Burger Joint,Asian Restaurant,Vietnamese Restaurant
81,Third Ward,Fast Food Restaurant,Frame Store,Burger Joint,Asian Restaurant,Vietnamese Restaurant
82,Third Ward,Fast Food Restaurant,Frame Store,Burger Joint,Asian Restaurant,Vietnamese Restaurant
83,Third Ward,Fast Food Restaurant,Frame Store,Burger Joint,Asian Restaurant,Vietnamese Restaurant
84,Third Ward,Fast Food Restaurant,Frame Store,Burger Joint,Asian Restaurant,Vietnamese Restaurant
85,Third Ward,Fast Food Restaurant,Frame Store,Burger Joint,Asian Restaurant,Vietnamese Restaurant
86,Third Ward,Fast Food Restaurant,Frame Store,Burger Joint,Asian Restaurant,Vietnamese Restaurant
87,Third Ward,Fast Food Restaurant,Frame Store,Burger Joint,Asian Restaurant,Vietnamese Restaurant
88,Third Ward,Fast Food Restaurant,Frame Store,Burger Joint,Asian Restaurant,Vietnamese Restaurant
89,Third Ward,Fast Food Restaurant,Frame Store,Burger Joint,Asian Restaurant,Vietnamese Restaurant


Cluster 5

In [247]:
Hou_wards_df_merged.loc[Hou_wards_df_merged['Cluster Labels'] == 4, Hou_wards_df_merged.columns[[0] + list(range(6, Hou_wards_df_merged.shape[1]))]]

,Ward,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
122,Hou Heights,Performing Arts Venue,Salon / Barbershop,Gift Shop,New American Restaurant,Pizza Place
123,Hou Heights,Performing Arts Venue,Salon / Barbershop,Gift Shop,New American Restaurant,Pizza Place
124,Hou Heights,Performing Arts Venue,Salon / Barbershop,Gift Shop,New American Restaurant,Pizza Place
125,Hou Heights,Performing Arts Venue,Salon / Barbershop,Gift Shop,New American Restaurant,Pizza Place
126,Hou Heights,Performing Arts Venue,Salon / Barbershop,Gift Shop,New American Restaurant,Pizza Place
127,Hou Heights,Performing Arts Venue,Salon / Barbershop,Gift Shop,New American Restaurant,Pizza Place
128,Hou Heights,Performing Arts Venue,Salon / Barbershop,Gift Shop,New American Restaurant,Pizza Place
129,Hou Heights,Performing Arts Venue,Salon / Barbershop,Gift Shop,New American Restaurant,Pizza Place
